In [2]:
import numpy as np
import PIL
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

#import cv2
import os
import random

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1318: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [11]:
# Directory of the training data
train_data_dir = '/home/thanhnt/phase_1/data'
working_dir = '/home/thanhnt/melanoma_tutorial'
"""
-data
    - img
    - label
    - label_zo
    - result
    - test
"""
root = '/home/thanhnt/phase_1'

# 1. Convert label data to binary

In [30]:
img_list = os.listdir(train_data_dir + '/label/')
label_colors = [(0,0,0),(255,255,255)]

In [32]:
for in_idx, in_ in enumerate(img_list):
    im = PIL.Image.open('%s/label/%s.png'%(train_data_dir,in_[:-4])).convert('RGB')
    im = np.array(im)
    # Convert to one channel binary image
    tmp = np.uint8(np.zeros(im[:,:,0:1].shape))
    for i in range(0,len(label_colors)):
        tmp[:,:,0] = tmp[:,:,0] + i*np.prod(np.equal(im,label_colors[i]),2)
    PIL.Image.fromarray(tmp[:,:,0]).save('%s/label_zo/%s.png'%(train_data_dir,in_[:-4]))
print 'train_label_Image done'

train_label_Image done


# 2. Prepare .txt

## Goal:
To prepare 'train_list.txt' and 'val_list.txt' where each line in the files hase the following format:  


```<path to a training image > <path to its binary label>```

In [21]:
# Reading all training names into a list
img_list = os.listdir(train_data_dir +'/img/')
img_list = [x[:-4] for x in img_list]
img_set = set(img_list)

print("#of training examples = {0:d}".format(len(img_set)))

#of training examples = 900


In [22]:
random.seed(2015)
random.shuffle(img_list)

In [7]:
# Divide the training data into: train (90%) and val (10%)
val_list = img_list[:int(len(img_set)*.1)]

In [8]:
val_set = set(val_list)
print(len(val_set))

90


In [17]:
f = open(working_dir + '/data/txt/val_list.txt', 'w')
f.write('\n'.join(['img/'+i+'.jpg label_zo/'+i+'_Segmentation.png' for i in val_list]))
f.close()

In [25]:
train_set = img_set - val_set
train_list = list(train_set)
train_list.sort()
print len(train_list)

810


In [26]:
f = open(working_dir + '/data/txt/train_list.txt', 'w')
f.write('\n'.join(['img/'+i+'.jpg label_zo/'+i+'_Segmentation.png' for i in train_list]))
f.close()

# 3. Compute mean
BGR mean of the training data

In [18]:
## naive approach
bgr_sum_mean = np.array([0., 0., 0.])
for in_ in train_list:
    img_orig_arr = np.array(PIL.Image.open('%s/img/%s.jpg'%(train_data_dir,in_)).convert('RGB'))
    img_orig_arr = img_orig_arr[:, :, ::-1]
    img_orig_arr = img_orig_arr.transpose((2, 0, 1))
    bgr_sum_mean += img_orig_arr.reshape(3, img_orig_arr.shape[1] * img_orig_arr.shape[2]).mean(axis = 1)

In [19]:
print 'BGR mean: ', bgr_sum_mean / (1. * len(train_list))

BGR mean:  [ 144.15102414  157.14571714  184.01073585]
